# 🧮 Logistic Regression — From First Principles

*Implement it from scratch, understand every formula, and see why it is **classification**, not regression.*

---

## 1. What Problem Does Logistic Regression Solve?

|         | Regression models | **Logistic regression** |
|---------|------------------|-------------------------|
| **Output** | A **continuous** real number (e.g. house price) | A **probability** in *(0, 1)* → hard‐class 0 / 1 |
| **Typical question** | “How much will the house cost?” | “Will the loan be **approved (1)** or **denied (0)**?” |
| **Goal** | Minimise distance between predicted & true numbers | Maximise probability assigned to correct class |

It is therefore a **binary classifier** even though the word *regression* lingers in its name.

---

## 2. Model Equation

<div align="center">

\[
\begin{aligned}
z &= \mathbf{w}^\top\mathbf{x} + b \\[4pt]
\hat{p} &= g(z) = \frac{1}{1+e^{-z}} \quad\text{(sigmoid)} \\[4pt]
\underbrace{\hat{y}}_{\text{hard label}}
&=\begin{cases}
1 &\text{if }\hat{p}\ge 0.5\\
0 &\text{otherwise}
\end{cases}
\end{aligned}
\]

![Sigmoid curve — outputs stay between 0 and 1, crossing 0.5 when z = 0.](sigmoid.png)

</div>

* **Linear score** \(z\) measures how far a point is from the decision boundary  
* **Sigmoid** squashes any real \(z\) into a value strictly between 0 and 1  
* **Decision boundary** is the hyper-plane where \(z = 0\) (because then \(g(z)=0.5\))
- **wᵀ (w-transpose)** – the weight column-vector turned sideways into a row.  
  *Purpose:* lets us compute the dot product `wᵀ x = w1·x1 + … + wk·xk`, which equals the linear score z.

- **ŷ_hard** (“hard hat-y”) – the crisp 0 / 1 label after thresholding.  
  *Rule:* if the probability p = g(z) is ≥ 0.5, set ŷ_hard = 1; otherwise 0.  
  Needed for metrics like accuracy that expect exact class labels.

- **Soft vs. hard outputs**  
  - **Soft output:** p = g(z) – a probability strictly between 0 and 1.  
  - **Hard output:** ŷ_hard – that same probability converted to 0 or 1 by the threshold rule.

---

## 3. Why Mean-Squared Error (MSE) **Breaks Down** for Logistic Regression  

### 3.1 — What if we naïvely reused MSE?

$$
C_{\text{MSE}}(\mathbf{w}, b)
  = \frac{1}{m} \sum_{i=1}^{m} \bigl(g(z_i) - y_i\bigr)^2,
\qquad
z_i = \mathbf{w}^{\top}\mathbf{x}^{(i)} + b,
\qquad
g(z) = \frac{1}{\,1 + e^{-z}}.
$$

### 3.2 — The problem in pictures  

| Linear regression | **Logistic + MSE** |
|-------------------|--------------------|
| ![smooth U-shape](linear.png) | ![many bumps](mse_bumpy.png) |
| One clean, convex “bowl”. | Plateaus at the edges, multiple little valleys (local minima). |

### 3.3 — Why do the bumps and plateaus appear with MSE?

1. **Sigmoid saturation (flat tails)**  

   $$
   g(z)=\frac{1}{1+e^{-z}},
   \qquad
   g'(z)=g(z)\bigl(1-g(z)\bigr).
   $$

   * If $z\ll 0$ ⇒ $g(z)\approx 0$ and **$g'(z)\approx 0$** – the left tail is almost flat.  
   * If $z\gg 0$ ⇒ $g(z)\approx 1$ and **$g'(z)\approx 0$** – the right tail is almost flat.  
   These flat regions are the **plateaus** at the edges of the cost surface.
   
2. **What is $g$ and why is $g'(z)=g(z)\,[1-g(z)]$ ?**

- **Sigmoid definition**

  $$
  g(z)=\frac{1}{\,1+e^{-z}\,}
  $$

- **Differentiate step-by-step**

  1. Rewrite  

     $$
     g(z)=\bigl(1+e^{-z}\bigr)^{-1}
     $$

  2. Apply the chain rule  

     $$
     \frac{d}{dz}\bigl(u^{-1}\bigr)=-u^{-2}\,u',
     \qquad
     u=1+e^{-z},\; u'=-e^{-z}.
     $$

  3. Simplify to obtain  

     $$
     g'(z)=\frac{e^{-z}}{\bigl(1+e^{-z}\bigr)^{2}}
           =g(z)\,\bigl[1-g(z)\bigr].
     $$


3. **Why does $x_j$ appear in the gradient $\partial E/\partial w_j$ ?**

- **One-sample squared error**

  $$
  E=\bigl(g(z)-y\bigr)^{2},
  \qquad
  z=\mathbf{w}^{\top}\mathbf{x}+b.
  $$

- **Chain-rule expansion**

  $$
  \frac{\partial E}{\partial w_j}
    =2\bigl(g(z)-y\bigr)\,
      g'(z)\,
      \underbrace{\frac{\partial z}{\partial w_j}}_{=\,x_j}.
  $$

- **Final form**

  $$
  \boxed{\displaystyle
  \frac{\partial E}{\partial w_j}
     =2\bigl(g(z)-y\bigr)\,g'(z)\,x_j}
  $$

  The feature value $x_j$ appears because changing weight $w_j$ shifts the linear score $z$ by exactly $x_j$; larger features exert a proportionally larger influence on the gradient.
  
4. **Squared error sees only distance, not direction**  

   For one sample  
   $$
   E \;=\; \bigl(g(z)-y\bigr)^2.
   $$  

   Gradient w.r.t. a weight $w_j$:  

   $$
   \frac{\partial E}{\partial w_j}
      = 2\bigl(g(z)-y\bigr)\;g'(z)\;x_j.
   $$  

   *On the plateaus* $g'(z)\!\approx\!0$, so the gradient almost vanishes even if the prediction is wrong — optimisation “stalls” on those flats.

5. **Opposing pulls from labels 0 and 1 create ripples**  

   * A point with $y=0$ wants $g(z)\!\downarrow$ (push $z$ left).  
   * A point with $y=1$ wants $g(z)\!\uparrow$ (push $z$ right).  

   Each class digs its own shallow valley in a different region.  
   Summing all those valleys produces the **multi-hump landscape** (many local minima).

> **Result:** Gradient descent can settle in a **shallow local dip** and stop, never reaching the true global minimum.

---

## 4 · Log-Loss (Cross-Entropy) — How It “Irons the Plateaus Flat”

For **each** training example the loss is  

$$
L(\hat{p},y)=
\begin{cases}
-\log(\hat{p})            & \text{if } y = 1,\\[6pt]
-\log\!\bigl(1-\hat{p}\bigr) & \text{if } y = 0.
\end{cases}
$$

* **$-\log(\hat{p})$** shoots upward as $\hat{p}\!\to 0$ &nbsp;→&nbsp; harshly punishes *confident-wrong* predictions for class 1.  
* **$-\log(1-\hat{p})$** shoots upward as $\hat{p}\!\to 1$ &nbsp;→&nbsp; harshly punishes *confident-wrong* predictions for class 0.  


| Loss curve for **$y = $**  (`- log(p)`) | Loss curve for **$y = $**  (`- log(1 – p)`) |
|------------------------------------------|----------------------------------------------|
| ![−log(p)](-logx.png) | ![−log(1 − p)](-log1-x.png) |


These logarithmic “walls” keep the gradient alive even on the sigmoid’s flat tails, so the many little valleys produced by MSE are **merged into one deep, convex bowl**.


### Average loss over the dataset  

$$
C(\mathbf{w},b)=
\frac{1}{m}\sum_{i=1}^{m}
\Bigl[-\,y_i\,\log\hat{p}_i\;-\;(1-y_i)\,\log\bigl(1-\hat{p}_i\bigr)\Bigr],
\qquad
\hat{p}_i = g\!\bigl(\mathbf{w}^{\top}\mathbf{x}^{(i)}+b\bigr).
$$

* **Correct & confident** ⟶ loss $\to 0$  
* **Wrong & confident** ⟶ loss $\to \infty$  
* The summed surface is always **smooth & convex** → gradient descent finds the single global minimum.  

---

## 5 · Gradient Descent — Fitting $w$ and $b$

> Iteratively slide **downhill** on the log-loss surface until you reach the single global bowl’s bottom.

### 5.1 Compute the gradients

For every training sample \(i\):

\[
\begin{aligned}
z_i &= \mathbf{w}^{\top}\mathbf{x}^{(i)} + b \\[4pt]
\hat{p}_i &= g(z_i)=\frac{1}{1+e^{-z_i}}
\end{aligned}
\]

Using the log-loss cost derived in Section&nbsp;4,

$$
C(\mathbf{w},b)
   =\frac{1}{m}\sum_{i=1}^{m}
     \Bigl[-\,y_i\,\log(\hat{p}_i)
           \;-\;(1-y_i)\,\log\!\bigl(1-\hat{p}_i\bigr)
     \Bigr],
\qquad
\hat{p}_i = g\!\bigl(\mathbf{w}^{\top}\mathbf{x}^{(i)} + b\bigr).
$$

**Partial derivatives**

$$
\boxed{\;
\frac{\partial C}{\partial w_j}
   = \frac{1}{m}\sum_{i=1}^{m}(\hat{p}_i - y_i)\,x_{i,j}
\;}
\qquad
\boxed{\;
\frac{\partial C}{\partial b}
   = \frac{1}{m}\sum_{i=1}^{m}(\hat{p}_i - y_i)
\;}
$$


*Reasoning:*  

$$
g'(z)=g(z)\,[1-g(z)]
\qquad\text{and}\qquad
\frac{\partial z_i}{\partial w_j}=x_{i,j}.
$$
